In [5]:
// Write a program to find the mutual friends between two users,given the input having two columns, user_id and friends_list.

val data = Seq(
    (1, Seq(2, 3, 4)),
    (2, Seq(1, 3, 4)),
    (3, Seq(1, 2)),
    (4, Seq(1, 2))
)

val schema = List("user_id", "friend_list")

val df = spark.createDataFrame(data).toDF(schema:_*)

df.show(false)

+-------+-----------+
|user_id|friend_list|
+-------+-----------+
|1      |[2, 3, 4]  |
|2      |[1, 3, 4]  |
|3      |[1, 2]     |
|4      |[1, 2]     |
+-------+-----------+



data: Seq[(Int, Seq[Int])] = List((1,List(2, 3, 4)), (2,List(1, 3, 4)), (3,List(1, 2)), (4,List(1, 2)))
schema: List[String] = List(user_id, friend_list)
df: org.apache.spark.sql.DataFrame = [user_id: int, friend_list: array<int>]


In [23]:
val pairs_df = df.as("f1").join(df.as("f2"), $"f1.user_id" =!= $"f2.user_id")

pairs_df.show(false)

+-------+-----------+-------+-----------+
|user_id|friend_list|user_id|friend_list|
+-------+-----------+-------+-----------+
|1      |[2, 3, 4]  |2      |[1, 3, 4]  |
|1      |[2, 3, 4]  |3      |[1, 2]     |
|1      |[2, 3, 4]  |4      |[1, 2]     |
|2      |[1, 3, 4]  |1      |[2, 3, 4]  |
|2      |[1, 3, 4]  |3      |[1, 2]     |
|2      |[1, 3, 4]  |4      |[1, 2]     |
|3      |[1, 2]     |1      |[2, 3, 4]  |
|3      |[1, 2]     |2      |[1, 3, 4]  |
|3      |[1, 2]     |4      |[1, 2]     |
|4      |[1, 2]     |1      |[2, 3, 4]  |
|4      |[1, 2]     |2      |[1, 3, 4]  |
|4      |[1, 2]     |3      |[1, 2]     |
+-------+-----------+-------+-----------+



pairs_df: org.apache.spark.sql.DataFrame = [user_id: int, friend_list: array<int> ... 2 more fields]


In [24]:
val mutual_friends_df = pairs_df.withColumn("mutual_friends", array_intersect($"f1.friend_list", $"f2.friend_list")
                                           ).select("f1.user_id", "f2.user_id", "mutual_friends")

mutual_friends_df.show(false)


+-------+-------+--------------+
|user_id|user_id|mutual_friends|
+-------+-------+--------------+
|1      |2      |[3, 4]        |
|1      |3      |[2]           |
|1      |4      |[2]           |
|2      |1      |[3, 4]        |
|2      |3      |[1]           |
|2      |4      |[1]           |
|3      |1      |[2]           |
|3      |2      |[1]           |
|3      |4      |[1, 2]        |
|4      |1      |[2]           |
|4      |2      |[1]           |
|4      |3      |[1, 2]        |
+-------+-------+--------------+



mutual_friends_df: org.apache.spark.sql.DataFrame = [user_id: int, user_id: int ... 1 more field]
